In [124]:
import numpy as np
import pandas as pd

# Configuracion para las librerias
pd.options.mode.chained_assignment = None
np.set_printoptions(precision = 3)

# Se carga el archivo
data = pd.read_csv("Covid Data.csv")

print(data)

         USMER  MEDICAL_UNIT  SEX  PATIENT_TYPE   DATE_DIED  INTUBED  \
0            2             1    1             1  03/05/2020       97   
1            2             1    2             1  03/06/2020       97   
2            2             1    2             2  09/06/2020        1   
3            2             1    1             1  12/06/2020       97   
4            2             1    2             1  21/06/2020       97   
...        ...           ...  ...           ...         ...      ...   
1048570      2            13    2             1  9999-99-99       97   
1048571      1            13    2             2  9999-99-99        2   
1048572      2            13    2             1  9999-99-99       97   
1048573      2            13    2             1  9999-99-99       97   
1048574      2            13    2             1  9999-99-99       97   

         PNEUMONIA  AGE  PREGNANT  DIABETES  ...  ASTHMA  INMSUPR  \
0                1   65         2         2  ...       2        2 

In [125]:

def normalize(xs):
    xs_norm = xs.copy()
    mu = np.zeros(xs.size)
    sigma = np.zeros(xs.size)
    mu = np.mean(xs, axis = 0)
    sigma = np.std(xs, axis = 0)
    xs_norm = (xs - mu) / sigma
    print("Dataset normalizado")
    return xs_norm, mu, sigma

In [126]:
print(data.shape)

y = data[data.columns[19]]
x1 = data.iloc[:, 0:4]
x2 = data.iloc[:, 5:19]
x3 = data[data.columns[20]]

y = np.array(y)

x1 = np.array(x1)
x2 = np.array(x2)
x3 = np.array(x3)
x3 = x3[...,np.newaxis]

x = np.concatenate([x1, x2, x3], axis=1)

x, mu, sigma = normalize(x)
x = np.concatenate([np.ones((x.shape[0], 1)), x], axis=1)

print(x.shape)
print(y.shape)

print(x)
print(y)

(1048575, 21)
Dataset normalizado
(1048575, 20)
(1048575,)
[[ 1.     0.763 -2.143 ... -0.05  -0.04   0.474]
 [ 1.     0.763 -2.143 ... -0.245 -0.04   0.474]
 [ 1.     0.763 -2.143 ... -0.05  -0.04  -2.106]
 ...
 [ 1.     0.763  1.08  ... -0.05  -0.04   0.474]
 [ 1.     0.763  1.08  ... -0.05  -0.04   0.474]
 [ 1.     0.763  1.08  ... -0.05  -0.04   0.474]]
[3 5 3 ... 7 7 7]


In [127]:
def relu(x):
  return np.maximum(0, x)

def reluPrime(x):
  return x > 0

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def linear(x):
    return x

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

def crossentropy(y, y_hat):
    logits = y_hat[np.arange(len(y_hat)),y]
    entropy = - logits + np.log(np.sum(np.exp(y_hat),axis=-1))
    return entropy.mean()

def grad_crossentropy(y, y_hat):
    answers = np.zeros_like(y_hat)
    answers[np.arange(len(y_hat)),y] = 1    
    return (- answers + softmax(y_hat)) / y_hat.shape[0]

In [128]:
class MLP():
  def __init__(self, D_in, H, D_out, loss, grad_loss, activation):
    # pesos de la capa 1
    self.w1, self.b1 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(D_in+H)),
                                  size=(D_in, H)), np.zeros(H)
    # pesos de la capa 2
    self.w2, self.b2 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(H+D_out)),
                                  size=(H, D_out)), np.zeros(D_out)
    self.ws = []
    # función de pérdida y derivada
    self.loss = loss
    self.grad_loss = grad_loss
    # función de activación
    self.activation = activation

  def __call__(self, x):
    # salida de la capa 1
    self.h_pre = np.dot(x, self.w1) + self.b1
    self.h = relu(self.h_pre)
    # salida del MLP
    y_hat = np.dot(self.h, self.w2) + self.b2 
    return self.activation(y_hat)
    
  def fit(self, X, Y, epochs = 100, lr = 0.001, batch_size=None, verbose=True, log_each=1):
    batch_size = len(X) if batch_size == None else batch_size
    batches = len(X) // batch_size
    l = []
    for e in range(1,epochs+1):     
        # Mini-Batch Gradient Descent
        _l = []
        for b in range(batches):
            # batch de datos
            x = X[b*batch_size:(b+1)*batch_size]
            y = Y[b*batch_size:(b+1)*batch_size] 
            # salida del perceptrón
            y_pred = self(x) 
            # función de pérdida
            loss = self.loss(y, y_pred)
            _l.append(loss)        
            # Backprop 
            dldy = self.grad_loss(y, y_pred) 
            grad_w2 = np.dot(self.h.T, dldy)
            grad_b2 = dldy.mean(axis=0)
            dldh = np.dot(dldy, self.w2.T)*reluPrime(self.h_pre)      
            grad_w1 = np.dot(x.T, dldh)
            grad_b1 = dldh.mean(axis=0)
            # Update (GD)
            self.w1 = self.w1 - lr * grad_w1
            self.b1 = self.b1 - lr * grad_b1
            self.w2 = self.w2 - lr * grad_w2
            self.b2 = self.b2 - lr * grad_b2
        l.append(np.mean(_l))
        # guardamos pesos intermedios para visualización
        self.ws.append((
            self.w1.copy(),
            self.b1.copy(),
            self.w2.copy(),
            self.b2.copy()
        ))
        if verbose and not e % log_each:
            print(f'Epoch: {e}/{epochs}, Loss: {np.mean(l):.5f}')

  def predict(self, ws, x):
    w1, b1, w2, b2 = ws
    h = relu(np.dot(x, w1) + b1)
    y_hat = np.dot(h, w2) + b2
    return self.activation(y_hat)

In [129]:
# MLP para clasificación multiclase
class MLPClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, crossentropy, grad_crossentropy, linear)

In [130]:
model = MLPClassification(D_in = 20, H = 50, D_out = 8)
epochs, lr = 20, 0.01
model.fit(x, y, epochs, lr, batch_size = 500, log_each = 1)

Epoch: 1/20, Loss: 0.61453
Epoch: 2/20, Loss: 0.58152
Epoch: 3/20, Loss: 0.56326
Epoch: 4/20, Loss: 0.55231
Epoch: 5/20, Loss: 0.54493
Epoch: 6/20, Loss: 0.53952
Epoch: 7/20, Loss: 0.53525
Epoch: 8/20, Loss: 0.53167
Epoch: 9/20, Loss: 0.52857
Epoch: 10/20, Loss: 0.52582
Epoch: 11/20, Loss: 0.52337
Epoch: 12/20, Loss: 0.52120
Epoch: 13/20, Loss: 0.51932
Epoch: 14/20, Loss: 0.51772
Epoch: 15/20, Loss: 0.51641
Epoch: 16/20, Loss: 0.51539
Epoch: 17/20, Loss: 0.51462
Epoch: 18/20, Loss: 0.51402
Epoch: 19/20, Loss: 0.51355
Epoch: 20/20, Loss: 0.51315


In [169]:
import random

ws = model.ws[-1]

p1 = random.randint(0, 1000000)
x1 = x[p1,:]
pred1 = model.predict(ws, x1)
pred1 = np.argmax(softmax(pred1))
real1 = y[p1]
print(f'El valor real es {real1}')
print('Predicción de la red neuronal: {}'.format(pred1))

p2 = random.randint(0, 1000000)
x2 = x[p2,:]
pred2 = model.predict(ws, x2)
pred2 = np.argmax(softmax(pred2))
real2 = y[p2]
print(f'El valor real es {real2}')
print('Predicción de la red neuronal: {}'.format(pred2))

p3 = random.randint(0, 1000000)
x3 = x[p3,:]
pred3 = model.predict(ws, x3)
pred3 = np.argmax(softmax(pred3))
real3 = y[p3]
print(f'El valor real es {real3}')
print('Predicción de la red neuronal: {}'.format(pred3))

El valor real es 7
Predicción de la red neuronal: 7
El valor real es 5
Predicción de la red neuronal: 7
El valor real es 7
Predicción de la red neuronal: 7
